# Генерация датасетов моделью ChatGPT 4.1

In [1]:
EXAMPLE_ID = 0
MODEL = 'gpt-4.1'
START = 1800
OUT_COUNT = 50
OUT_FILE_PREFIX = 'chatgpt_essays'
PAUSE_SECONDS = 1

### Исходный датасет, написанный человеком

In [2]:
import pandas as pd

df = pd.read_csv("../datasets/human_essays.csv", encoding="utf-8")
df.head(10)

,title,text
0,12 Years a Slave: An Analysis of the Film Essay,The 2013 film 12 Years a Slave proved that sla...
1,20+ Social Media Post Ideas to Radically Simpl...,Social Media Examiner’s (2021) video on social...
2,533 U.S. 27 (2001) Kyllo v. United States: The...,Table of Contents\n 1. Facts\n 2. Issue\n 3. H...
3,A Charles Schwab Corporation Case Essay,Charles Schwab is a for-profit Corporation who...
4,A Clinical Office Assistant’s Attire Research ...,The work of a clinical or medical office worke...
5,A Comic Science Fiction Film “Back to the Futu...,Table of Contents\n 1. Introduction\n 2. Movie...
6,A Community Yard Sale as a Memorable Event Essay,Autumn is not generally viewed as an appropria...
7,A Complex in the “Every Secret Thing” Film by ...,Carl Jung identifies the mother complex with a...
8,A Customer Told to Wear a Mask Threw Snow at a...,The article describes a recent Chicago hot dog...
9,A Disconnect Between Public Transportation and...,One realistic way to commute is using private ...


#### Пример:

In [3]:
print(f"{df.iloc[EXAMPLE_ID,0]}\n\n{df.iloc[EXAMPLE_ID,1]}")

12 Years a Slave: An Analysis of the Film Essay

The 2013 film 12 Years a Slave proved that slavery is a worldwide issue. Indeed, the film made $150 million outside the United States and $57 million in the U.S., with a production budget of $20 million (Sharf, 2020). The movie was based on the memoir Twelve Years a Slave by Solomon Northup (Ntim, 2020). It tells the story of a free African American man who was kidnapped and sold into slavery. Solomon spent twelve years away from his family, being traded from one master to another. Fortunately, the protagonist met a person who helped him deliver a message to his family and friends, who came and rescued him. This movie accurately illustrates discriminatory relationships between white slaveholders and black slaves that stemmed from the dysfunctional system in the country and prejudices in people’s mindsets at that time.

The two main ethnic groups presented in this film are White and African Americans, and the three social groups are afflu

### Утилиты

In [4]:
import math
import re


def round_word_count(c):
    return math.floor(c / 100) * 100

def word_count(text):
    words = re.findall(r'\b\w+\b', text)
    return round_word_count(len(words))

def print_example(data, example_id):
    if 'prompt' in data.iloc[example_id]:
        print(f"Prompt {'=' * 93}\n{data.iloc[example_id]['prompt']}")
        print(f"{'=' * 100}")
        print(f"{data.iloc[example_id]['title']}\n\n{data.iloc[example_id]['text']}")
        print(f"{'=' * 100}")
    if 'prompt_2' in data.iloc[example_id]:
        print(f"Prompt 1 {'=' * 93}\n{data.iloc[example_id]['prompt_1']}")
        print(f"{'=' * 100}")
        print(f"{data.iloc[example_id]['response']}")
        print(f"Prompt 2 {'=' * 93}\n{data.iloc[example_id]['prompt_2']}")
        print(f"{'=' * 100}")
        print(f"{data.iloc[example_id]['title']}\n\n{data.iloc[example_id]['text']}")
        print(f"{'=' * 100}")

def print_progress(start, i, n):
    if (i - start) % (n / 100) == 0:
        print(f"{100 * (i - start) / n}% done")

In [5]:
from dotenv import load_dotenv
import os
from openai import OpenAI


load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [6]:
import time


def generate_response(prompt):
    response = client.responses.create(
        model=MODEL,
        input=prompt
    )

    time.sleep(PAUSE_SECONDS)
    
    return response.output_text

### Вариант 1. Prompt-based generation

```
Write an essay of about {} words on the topic '{}'.
Start directly with the first sentence of the essay. Absolutely no title or heading of any kind. 
Output only the essay text.
```

In [7]:
def generate(model_name, df, prompt_raw, start=0, n=5000):
    rows = []
    for i in range(start, start + n):
        title = df.loc[i, "title"]
        length = word_count(df.loc[i, "text"])
    
        prompt_formatted = prompt_raw.format(length, title)

        try:
            response = generate_response(prompt_formatted)
        except:
            continue
            
        response = response.lstrip()
    
        rows.append({"title": title, "prompt": prompt_formatted, "text": response, "model": model_name})

        print_progress(start, i, n)
    
    return pd.DataFrame(rows)

In [8]:
prompt = """
Write an essay of about {} words on the topic '{}'.
Start directly with the first sentence of the essay. Absolutely no title or heading of any kind. 
Output only the essay text. 
"""

df1 = generate(MODEL, df, prompt, start=START, n=OUT_COUNT)
df1.head()

0.0% done
2.0% done
4.0% done
6.0% done
8.0% done
10.0% done
12.0% done
14.0% done
16.0% done
18.0% done
20.0% done
22.0% done
24.0% done
26.0% done
28.0% done
30.0% done
32.0% done
34.0% done
36.0% done
38.0% done
40.0% done
42.0% done
44.0% done
46.0% done
48.0% done
50.0% done
52.0% done
54.0% done
56.0% done
58.0% done
60.0% done
62.0% done
64.0% done
66.0% done
68.0% done
70.0% done
72.0% done
74.0% done
76.0% done
78.0% done
80.0% done
82.0% done
84.0% done
86.0% done
88.0% done
90.0% done
92.0% done
94.0% done
96.0% done
98.0% done


,title,prompt,text,model
0,The Exponential Function in Analytic Algebra E...,\nWrite an essay of about 400 words on the top...,The exponential function stands as one of the ...,gpt-4.1
1,The Family Institution and Impact of Polygamy ...,\nWrite an essay of about 1600 words on the to...,The family has long been considered the corner...,gpt-4.1
2,The Family Voices Organization’s Mission & Ser...,\nWrite an essay of about 200 words on the top...,The Family Voices Organization is dedicated to...,gpt-4.1
3,The Film “The Big Lebowski” by Cohen Brothers ...,\nWrite an essay of about 2800 words on the to...,"The film ""The Big Lebowski,"" directed by Joel ...",gpt-4.1
4,“The First Americans” Book by Adovasio & Page ...,\nWrite an essay of about 600 words on the top...,James Adovasio and Jake Page’s book “The First...,gpt-4.1


#### Пример:

In [9]:
print_example(df1, EXAMPLE_ID)

Prompt =============================================================================================

Write an essay of about 400 words on the topic 'The Exponential Function in Analytic Algebra Essay'.
Start directly with the first sentence of the essay. Absolutely no title or heading of any kind. 
Output only the essay text. 

The Exponential Function in Analytic Algebra Essay

The exponential function stands as one of the foundational pillars in analytic algebra, weaving through diverse areas such as calculus, complex analysis, and differential equations. Its simplest form, \( f(x) = e^x \), encapsulates a profound mathematical concept: growth that is proportional to the current value. Unlike linear or polynomial functions, the exponential function exhibits a rate of increase that constantly accelerates, making it indispensable for modeling phenomena like compound interest, population dynamics, and radioactive decay.

In the framework of analytic algebra, the elegance of the exponen

In [10]:
file_path = f"../datasets/{OUT_FILE_PREFIX}_1.csv"

df1.to_csv(
    file_path, 
    mode="a",
    header=not os.path.exists(file_path),
    index=False,
    encoding="utf-8",
)

### Вариант 2. Prompt-based Style Transfer

```
Write an essay of about {} words on the topic '{}', following the stylistic and structural conventions of IvyPanda essays.

STYLE REQUIREMENTS (IvyPanda-like):
1. The tone must be academic, clear, neutral, and objective.
2. The essay should include background context, a clear analytical structure, and well-developed paragraphs.
3. Arguments must be supported with reasoning, concise explanations, and relevant examples.
4. The writing must be coherent, logically progressive, and easy to read.
5. Avoid emotional language or overly dramatic phrasing.

STRUCTURE REQUIREMENTS:
1. Divide the essay into several sections, each starting with a plain-text subheading.
2. Subheadings must contain only letters and spaces — no punctuation, no symbols, and absolutely no Markdown formatting.
   - This means: no asterisks, no bold, no italics, no underscores, no brackets.
   - Subheadings must appear as plain text lines, e.g. Background Context
3. Each subheading must be on its own line with exactly one blank line before and after it.
4. No overall essay title — begin directly with the first subheading.
5. Output must be plain text only (no Markdown, no asterisks, no bold, no italics, no hashtags, no special formatting).

OPTIONAL ELEMENTS:
You may include a plain-text references section at the end if appropriate to the topic, but this is optional.

Begin with the first subheading, then continue with the first paragraph.
Output only the essay text.
```

In [11]:
def generate(model_name, df, prompt_raw, start=0, n=5000):
    rows = []
    for i in range(start, start + n):
        title = df.loc[i, "title"]
        length = word_count(df.loc[i, "text"])
    
        prompt_formatted = prompt_raw.format(length, title)
    
        try:
            response = generate_response(prompt_formatted)
        except:
            continue
            
        response = response.lstrip()
    
        rows.append({"title": title, "prompt": prompt_formatted, "text": response, "model": model_name})

        print_progress(start, i, n)
    
    return pd.DataFrame(rows)

In [12]:
prompt = """
Write an essay of about {} words on the topic '{}', following the stylistic and structural conventions of IvyPanda essays.

STYLE REQUIREMENTS (IvyPanda-like):
1. The tone must be academic, clear, neutral, and objective.
2. The essay should include background context, a clear analytical structure, and well-developed paragraphs.
3. Arguments must be supported with reasoning, concise explanations, and relevant examples.
4. The writing must be coherent, logically progressive, and easy to read.
5. Avoid emotional language or overly dramatic phrasing.

STRUCTURE REQUIREMENTS:
1. Divide the essay into several sections, each starting with a plain-text subheading.
2. Subheadings must contain only letters and spaces — no punctuation, no symbols, and absolutely no Markdown formatting.
   - This means: no asterisks, no bold, no italics, no underscores, no brackets.
   - Subheadings must appear as plain text lines, e.g. Background Context
3. Each subheading must be on its own line with exactly one blank line before and after it.
4. No overall essay title — begin directly with the first subheading.
5. Output must be plain text only (no Markdown, no asterisks, no bold, no italics, no hashtags, no special formatting).

OPTIONAL ELEMENTS:
You may include a plain-text references section at the end if appropriate to the topic, but this is optional.

Begin with the first subheading, then continue with the first paragraph.
Output only the essay text.
"""

df2 = generate(MODEL, df, prompt, start=START, n=OUT_COUNT)
df2.head()

0.0% done
2.0% done
4.0% done
6.0% done
8.0% done
10.0% done
12.0% done
14.0% done
16.0% done
18.0% done
20.0% done
22.0% done
24.0% done
26.0% done
28.0% done
30.0% done
32.0% done
34.0% done
36.0% done
38.0% done
40.0% done
42.0% done
44.0% done
46.0% done
48.0% done
50.0% done
52.0% done
54.0% done
56.0% done
58.0% done
60.0% done
62.0% done
64.0% done
66.0% done
68.0% done
70.0% done
72.0% done
74.0% done
76.0% done
78.0% done
80.0% done
82.0% done
84.0% done
86.0% done
88.0% done
90.0% done
92.0% done
94.0% done
96.0% done
98.0% done


,title,prompt,text,model
0,The Exponential Function in Analytic Algebra E...,\nWrite an essay of about 400 words on the top...,Background Context\n\nThe exponential function...,gpt-4.1
1,The Family Institution and Impact of Polygamy ...,\nWrite an essay of about 1600 words on the to...,Background Context\n\nThe family institution i...,gpt-4.1
2,The Family Voices Organization’s Mission & Ser...,\nWrite an essay of about 200 words on the top...,Background Context\n\nFamily Voices is a natio...,gpt-4.1
3,The Film “The Big Lebowski” by Cohen Brothers ...,\nWrite an essay of about 2800 words on the to...,"Background Context\n\nReleased in 1998, “The B...",gpt-4.1
4,“The First Americans” Book by Adovasio & Page ...,\nWrite an essay of about 600 words on the top...,Background Context\n\nThe question of who the ...,gpt-4.1


#### Пример:

In [13]:
print_example(df2, EXAMPLE_ID)

Prompt =============================================================================================

Write an essay of about 400 words on the topic 'The Exponential Function in Analytic Algebra Essay', following the stylistic and structural conventions of IvyPanda essays.

STYLE REQUIREMENTS (IvyPanda-like):
1. The tone must be academic, clear, neutral, and objective.
2. The essay should include background context, a clear analytical structure, and well-developed paragraphs.
3. Arguments must be supported with reasoning, concise explanations, and relevant examples.
4. The writing must be coherent, logically progressive, and easy to read.
5. Avoid emotional language or overly dramatic phrasing.

STRUCTURE REQUIREMENTS:
1. Divide the essay into several sections, each starting with a plain-text subheading.
2. Subheadings must contain only letters and spaces — no punctuation, no symbols, and absolutely no Markdown formatting.
   - This means: no asterisks, no bold, no italics, no undersco

In [14]:
file_path = f"../datasets/{OUT_FILE_PREFIX}_2.csv"

df2.to_csv(
    file_path, 
    mode="a",
    header=not os.path.exists(file_path),
    index=False,
    encoding="utf-8",
)

### Вариант 3. Few-shot prompting

```
You are a helpful writing assistant. Your task is to generate essays of a given length and topic.
Start directly with the first sentence. No titles or headings. Output only the essay text.

Here is an example:

Example topic: "{}"
Example length: {} words

Example output:
{}

Now generate a new essay.

Topic: "{}"
Length: {} words

Write the essay now.
```

В качестве примера используется эссе из этого же набора, но на другую тему.

In [15]:
def generate_few_shot(model_name, df, prompt_raw, start=0, n=5000):
    rows = []
    for i in range(start + 1, start + n + 1):
        prev_title = df.loc[i - 1, "title"]
        prev_length = word_count(df.loc[i - 1, "text"])
        prev_text = df.loc[i - 1, "text"]
        
        title = df.loc[i, "title"]
        length = word_count(df.loc[i, "text"])
    
        prompt_formatted = prompt_raw.format(prev_title, prev_length, prev_text, title, length)

        try:
            response = generate_response(prompt_formatted)
        except:
            continue
            
        response = response.lstrip()
    
        rows.append({"title": title, "prompt": prompt_formatted, "text": response, "model": model_name})

        print_progress(start, i, n)
    
    return pd.DataFrame(rows)

In [16]:
prompt = """
You are a helpful writing assistant. Your task is to generate essays of a given length and topic.
Start directly with the first sentence. No titles or headings. Output only the essay text.

Here is an example:

Example topic: "{}"
Example length: {} words

Example output:
{}

Now generate a new essay.

Topic: "{}"
Length: {} words

Write the essay now.
"""

df3 = generate_few_shot(MODEL, df, prompt, start=START, n=OUT_COUNT)
df3.head()

2.0% done
4.0% done
6.0% done
8.0% done
10.0% done
12.0% done
14.0% done
16.0% done
18.0% done
20.0% done
22.0% done
24.0% done
26.0% done
28.0% done
30.0% done
32.0% done
34.0% done
36.0% done
38.0% done
40.0% done
42.0% done
44.0% done
46.0% done
48.0% done
50.0% done
52.0% done
54.0% done
56.0% done
58.0% done
60.0% done
62.0% done
64.0% done
66.0% done
68.0% done
70.0% done
72.0% done
74.0% done
76.0% done
78.0% done
80.0% done
82.0% done
84.0% done
86.0% done
88.0% done
90.0% done
92.0% done
94.0% done
96.0% done
98.0% done
100.0% done


,title,prompt,text,model
0,The Family Institution and Impact of Polygamy ...,\nYou are a helpful writing assistant. Your ta...,The family institution is a foundational struc...,gpt-4.1
1,The Family Voices Organization’s Mission & Ser...,\nYou are a helpful writing assistant. Your ta...,The Family Voices Organization is a national n...,gpt-4.1
2,The Film “The Big Lebowski” by Cohen Brothers ...,\nYou are a helpful writing assistant. Your ta...,"Joel and Ethan Coen’s “The Big Lebowski,” rele...",gpt-4.1
3,“The First Americans” Book by Adovasio & Page ...,\nYou are a helpful writing assistant. Your ta...,James M. Adovasio and Jake Page’s “The First A...,gpt-4.1
4,The Foundations of the Modern World Essay,\nYou are a helpful writing assistant. Your ta...,The foundations of the modern world rest on a ...,gpt-4.1


#### Пример:

In [17]:
print_example(df3, EXAMPLE_ID)

Prompt =============================================================================================

You are a helpful writing assistant. Your task is to generate essays of a given length and topic.
Start directly with the first sentence. No titles or headings. Output only the essay text.

Here is an example:

Example topic: "The Exponential Function in Analytic Algebra Essay"
Example length: 400 words

Example output:
The exponential function is essential in analytic algebra and is widely used in all related fields. It is unknown when man first began using exponential laws, but the first documented appearance of the exponent in printed works dates back to 1544. Exponents were written about in the book Arithmetic Integra by the German monk Michael Stifel (Reich, 2017). However, it is essential to say that Stifel did not use other bases for exponentiation than two, which means that there was no real exponentiation on the part of the German. Exponents are thought to have been found in t

In [18]:
file_path = f"../datasets/{OUT_FILE_PREFIX}_3.csv"

df3.to_csv(
    file_path, 
    mode="a",
    header=not os.path.exists(file_path),
    index=False,
    encoding="utf-8",
)

### Вариант 4. Content Abstraction (Regeneration Learning)

Первый промпт:
```
Identify the writing style and format of the text, excluding all semantic content and themes.

Text:
{}
```

Второй промпт:
```
Write an essay of about {} words on the topic '{}'.

You are given the following style-and-format description:
'{}'

Start directly with the first sentence of the essay. Absolutely no title or heading of any kind.
Output only the essay text.
```

In [19]:
def generate_with_content(model_name, df, prompt_raw_1, prompt_raw_2, start=0, n=5000):
    rows = []
    for i in range(start, start + n):
        title = df.loc[i, "title"]
        length = word_count(df.loc[i, "text"])
        text = df.loc[i, "text"]

        prompt_1_formatted = prompt_raw_1.format(text)

        try:
            response_1 = generate_response(prompt_1_formatted)
            response_1 = response_1.lstrip()
    
            prompt_2_formatted = prompt_raw_2.format(length, title, response_1)
            response_2 = generate_response(prompt_2_formatted)
        except:
            continue
            
        response = response_2.lstrip()
    
        rows.append({
            "title": title, 
            "prompt_1": prompt_1_formatted, 
            "response": response_1,
            "prompt_2": prompt_2_formatted, 
            "text": response, 
            "model": model_name
        })

        print_progress(start, i, n)
    
    return pd.DataFrame(rows)

In [20]:
prompt_1 = """
Identify the writing style and format of the text, excluding all semantic content and themes.

Text:
{}
"""

prompt_2 = """
Write an essay of about {} words on the topic '{}'.

You are given the following style-and-format description:
'{}'

Start directly with the first sentence of the essay. Absolutely no title or heading of any kind.
Output only the essay text.
"""

In [21]:
df4 = generate_with_content(MODEL, df, prompt_1, prompt_2, start=START, n=OUT_COUNT)
df4.head()

0.0% done
2.0% done
4.0% done
6.0% done
8.0% done
10.0% done
12.0% done
14.0% done
16.0% done
18.0% done
20.0% done
22.0% done
24.0% done
26.0% done
28.0% done
30.0% done
32.0% done
34.0% done
36.0% done
38.0% done
40.0% done
42.0% done
44.0% done
46.0% done
48.0% done
50.0% done
52.0% done
54.0% done
56.0% done
58.0% done
60.0% done
62.0% done
64.0% done
66.0% done
68.0% done
70.0% done
72.0% done
74.0% done
76.0% done
78.0% done
80.0% done
82.0% done
84.0% done
86.0% done
88.0% done
90.0% done
92.0% done
94.0% done
96.0% done
98.0% done


,title,prompt_1,response,prompt_2,text,model
0,The Exponential Function in Analytic Algebra E...,\nIdentify the writing style and format of the...,Certainly! Here is the analysis of the **writi...,\nWrite an essay of about 400 words on the top...,"The exponential function, a pivotal construct ...",gpt-4.1
1,The Family Institution and Impact of Polygamy ...,\nIdentify the writing style and format of the...,The **writing style** of the text is **academi...,\nWrite an essay of about 1600 words on the to...,The family institution serves as a fundamental...,gpt-4.1
2,The Family Voices Organization’s Mission & Ser...,\nIdentify the writing style and format of the...,**Writing Style:**\n- **Objective and Informat...,\nWrite an essay of about 200 words on the top...,Family Voices is a national organization estab...,gpt-4.1
3,The Film “The Big Lebowski” by Cohen Brothers ...,\nIdentify the writing style and format of the...,**Writing Style:** \nThe writing style is **a...,\nWrite an essay of about 2800 words on the to...,The cinematic oeuvre of Joel and Ethan Coen ha...,gpt-4.1
4,“The First Americans” Book by Adovasio & Page ...,\nIdentify the writing style and format of the...,Certainly! Below is an analysis of the **writi...,\nWrite an essay of about 600 words on the top...,James M. Adovasio and Jake Page’s “The First A...,gpt-4.1


#### Пример:

In [22]:
print_example(df4, EXAMPLE_ID)

Prompt 1 =============================================================================================

Identify the writing style and format of the text, excluding all semantic content and themes.

Text:
The exponential function is essential in analytic algebra and is widely used in all related fields. It is unknown when man first began using exponential laws, but the first documented appearance of the exponent in printed works dates back to 1544. Exponents were written about in the book Arithmetic Integra by the German monk Michael Stifel (Reich, 2017). However, it is essential to say that Stifel did not use other bases for exponentiation than two, which means that there was no real exponentiation on the part of the German. Exponents are thought to have been found in the Sumerian language of ancient Babylon as well (Smith, 2017). However, because of the qualitative difference in the use of the mathematical alphabet, the Babylonian exponentiation was different from what is used today.

In [23]:
file_path = f"../datasets/{OUT_FILE_PREFIX}_4.csv"

df4.to_csv(
    file_path, 
    mode="a",
    header=not os.path.exists(file_path),
    index=False,
    encoding="utf-8",
)

### Вариант 5. Outline‑guided Text Generation

Первый промпт:
```
You are an academic writing instructor.
Create a well-structured outline for an academic essay on the following topic: {}.

Requirements:

The outline must include the following sections:
- Introduction
- Main Body (2–4 thematic sections)
- Counterarguments / Alternative Perspectives
- Conclusion

Each section should contain 2–3 concise bullet points describing the key ideas or arguments.
The outline should reflect a clear thesis-driven structure.
Use formal academic language.

Do not write the essay itself
```

Второй промпт:
```
You are an academic essay writer.
Write a coherent academic essay of about {} words on the topic '{}' strictly following the outline below.

Outline:
{}

Requirements:
- Follow the outline in the given order without adding or removing sections.
- Develop each bullet point into a full, well-argued paragraph.
- Maintain a formal academic tone and clear argumentative logic.
- Use appropriate academic transitions and signposting.
- Do not introduce new arguments or themes not present in the outline.
- Avoid informal language, personal anecdotes, and unsupported claims.
```

In [24]:
def generate_with_plan(model_name, df, prompt_raw_1, prompt_raw_2, start=0, n=5000):
    rows = []
    for i in range(start, start + n):
        title = df.loc[i, "title"]
        length = word_count(df.loc[i, "text"])
        text = df.loc[i, "text"]
    
        prompt_1_formatted = prompt_raw_1.format(title)

        try:
            response_1 = generate_response(prompt_1_formatted)
            response_1 = response_1.lstrip()
    
            prompt_2_formatted = prompt_raw_2.format(length, title, response_1)
            response_2 = generate_response(prompt_2_formatted)
        except:
            continue
            
        response = response_2.lstrip()
    
        rows.append({
            "title": title, 
            "prompt_1": prompt_1_formatted, 
            "response": response_1,
            "prompt_2": prompt_2_formatted, 
            "text": response, 
            "model": model_name
        })

        print_progress(start, i, n)
    
    return pd.DataFrame(rows)

In [25]:
prompt_1 = """
You are an academic writing instructor.
Create a well-structured outline for an academic essay on the following topic: {}.

Requirements:

The outline must include the following sections:
- Introduction
- Main Body (2–4 thematic sections)
- Counterarguments / Alternative Perspectives
- Conclusion

Each section should contain 2–3 concise bullet points describing the key ideas or arguments.
The outline should reflect a clear thesis-driven structure.
Use formal academic language.

Do not write the essay itself
"""

prompt_2 = """
You are an academic essay writer.
Write a coherent academic essay of about {} words on the topic '{}' strictly following the outline below.

Outline:
{}

Requirements:
- Follow the outline in the given order without adding or removing sections.
- Develop each bullet point into a full, well-argued paragraph.
- Maintain a formal academic tone and clear argumentative logic.
- Use appropriate academic transitions and signposting.
- Do not introduce new arguments or themes not present in the outline.
- Avoid informal language, personal anecdotes, and unsupported claims.
"""

In [ ]:
df5 = generate_with_plan(MODEL, df, prompt_1, prompt_2, start=START, n=OUT_COUNT)
df5.head()

0.0% done
2.0% done


#### Пример:

In [ ]:
print_example(df5, EXAMPLE_ID)

In [ ]:
file_path = f"../datasets/{OUT_FILE_PREFIX}_5.csv"

df5.to_csv(
    file_path, 
    mode="a",
    header=not os.path.exists(file_path),
    index=False,
    encoding="utf-8",
)